# Instalando o Kaggle

In [1]:
!pip install kaggle

# Realizando o upload do token de segurança de comunicação com o kaggle

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 72 bytes


In [ ]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        153           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        289           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        851           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       5165           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      40678           False  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      10678           False  
connectx

# Fazendo o download do dataset do kaggle

In [3]:
!kaggle datasets download -d saraivaufc/automatic-weather-stations-brazil

 99% 1.39G/1.40G [00:12<00:00, 140MB/s]
100% 1.40G/1.40G [00:12<00:00, 123MB/s]


# Descompactando o dataset baixado

In [4]:
!unzip automatic-weather-stations-brazil.zip

Archive:  automatic-weather-stations-brazil.zip
  inflating: automatic_stations_codes_2000_2021.csv  
  inflating: automatic_weather_stations_inmet_brazil_2000_2021.csv  


# Configurando o Spark

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop3.2.tgz
!tar xf spark-3.0.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop3.2"

import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

In [7]:
# Criando uma sessão no Spark
spark = SparkSession.builder.getOrCreate()

from datetime import datetime, date  
from pyspark.sql import Row

Carregando dataset no spark

In [8]:
linhas = spark.sparkContext.textFile('automatic_weather_stations_inmet_brazil_2000_2021.csv')

In [9]:
df = linhas.map(lambda line:line.split(';'))

print(df)

#Colunas desejadas
def func(lista):
  return lista[0], lista[1], lista[2], lista[3], lista[4], lista[8], lista[16], lista[19]
 
df_limpo = df.map(func) 

PythonRDD[2] at RDD at PythonRDD.scala:53


In [10]:
df_limpo.take(2)


[('ESTACAO',
  'DATA (YYYY-MM-DD)',
  'HORA (UTC)',
  'PRECIPITACAO TOTAL HORARIO (mm)',
  'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
  'TEMPERATURA DO AR - BULBO SECO, HORARIA (C)',
  'UMIDADE RELATIVA DO AR, HORARIA (%)',
  'VENTO, VELOCIDADE HORARIA (m/s)'),
 ('A001', '2021-01-01', '0000', '0.0', '887.1', '20.9', '73.0', '0.8')]

In [ ]:
#tentar otimizar filtro com isso aqui
filter = ['A306','A314','A315','A319','A324','A325','A332','A339','A342','A347','A358','A359','A360','A368','A369']

In [11]:
df_ceara = df_limpo.filter(lambda x: 'A306' in x or 'A314' in x or 'A315' in x or 'A319' in x or 'A324' in x or 'A325' in x or 'A332' in x or 'A339' in x or 'A342' in x or 'A347' in x or 'A358' in x or 'A359' in x or 'A360' in x or 'A368' in x or 'A369' in x).take(50000)

#df_ceara = df_ceara.map(lambda line:line.split('['))



In [12]:
len(df_ceara)

50000

In [79]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DateType, FloatType, TimestampType

schema = StructType([
    StructField('ESTACAO', StringType(), True),
    StructField('DATA', DateType(), True),
    StructField('HORA', TimestampType(), True),
    StructField('PRECIPITACAO TOTAL HORARIO (mm)', IntegerType(), True),
    StructField('PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', FloatType(), True),
    StructField('TEMPERATURA DO AR - BULBO SECO, HORARIA (C)', FloatType(), True),
    StructField('UMIDADE RELATIVA DO AR, HORARIA (%)', FloatType(), True),
    StructField('VENTO, VELOCIDADE HORARIA (m/s)', FloatType(), True),
])

# Convert list to RDD
rdd = spark.sparkContext.parallelize(df_ceara)
df = spark.createDataFrame(rdd,['ESTACAO',
  'DATA',
  'HORA',
  'PRECIPITACAO',
  'PRESSAO_ATMOSFERICA',
  'TEMPERATURA',
  'UMIDADE',
  'VENTO'])

# Create data frame
#df = spark.createDataFrame(rdd,schema)
#print(df.schema)
df.show()

+-------+----------+----+------------+-------------------+-----------+-------+-----+
|ESTACAO|      DATA|HORA|PRECIPITACAO|PRESSAO_ATMOSFERICA|TEMPERATURA|UMIDADE|VENTO|
+-------+----------+----+------------+-------------------+-----------+-------+-----+
|   A306|2021-01-01|0000|         0.0|             1000.1|       27.7|   65.0|  3.4|
|   A306|2021-01-01|0100|         0.0|             1000.9|       27.0|   74.0|  0.6|
|   A306|2021-01-01|0200|         0.0|             1001.5|       26.3|   79.0|  0.0|
|   A306|2021-01-01|0300|         0.0|             1001.2|       26.1|   82.0|  0.0|
|   A306|2021-01-01|0400|         0.0|             1000.7|       26.0|   84.0|  0.3|
|   A306|2021-01-01|0500|         0.0|             1000.3|       25.6|   84.0|  0.0|
|   A306|2021-01-01|0600|         0.0|             1000.0|       25.1|   87.0|  0.0|
|   A306|2021-01-01|0700|         0.0|             1000.1|       24.8|   88.0|  0.0|
|   A306|2021-01-01|0800|         0.0|             1000.9|       

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.createOrReplaceTempView('tableA')
print(spark.sql('select * from tableA').show())

+-------+----------+----+------------+-------------------+-----------+-------+-----+
|ESTACAO|      DATA|HORA|PRECIPITACAO|PRESSAO_ATMOSFERICA|TEMPERATURA|UMIDADE|VENTO|
+-------+----------+----+------------+-------------------+-----------+-------+-----+
|   A306|2021-01-01|0000|         0.0|             1000.1|       27.7|   65.0|  3.4|
|   A306|2021-01-01|0100|         0.0|             1000.9|       27.0|   74.0|  0.6|
|   A306|2021-01-01|0200|         0.0|             1001.5|       26.3|   79.0|  0.0|
|   A306|2021-01-01|0300|         0.0|             1001.2|       26.1|   82.0|  0.0|
|   A306|2021-01-01|0400|         0.0|             1000.7|       26.0|   84.0|  0.3|
|   A306|2021-01-01|0500|         0.0|             1000.3|       25.6|   84.0|  0.0|
|   A306|2021-01-01|0600|         0.0|             1000.0|       25.1|   87.0|  0.0|
|   A306|2021-01-01|0700|         0.0|             1000.1|       24.8|   88.0|  0.0|
|   A306|2021-01-01|0800|         0.0|             1000.9|       

#Importando Bibliotecas: Data analysis

In [14]:
import numpy as np
import random as rnd
from pyspark.sql.window import Window
import pyspark.sql.functions as F


# Analise Descritiva dos Dados

In [15]:
df.describe().show()

+-------+-------+----------+-----------------+-------------------+-------------------+------------------+-----------------+------------------+
|summary|ESTACAO|      DATA|             HORA|       PRECIPITACAO|PRESSAO_ATMOSFERICA|       TEMPERATURA|          UMIDADE|             VENTO|
+-------+-------+----------+-----------------+-------------------+-------------------+------------------+-----------------+------------------+
|  count|  50000|     50000|            50000|              50000|              50000|             50000|            50000|             50000|
|   mean|   null|      null|         1149.872|0.10227919356319039|  976.9155263829372|26.625177204423082| 66.7407228605452|1.8212265622783852|
| stddev|   null|      null|692.2502205296557| 1.0429634475200882|    24.179291972851| 4.001085447165693|19.95580993809345|1.5012775504691624|
|    min|   A306|2020-01-01|                0|                   |                   |                  |                 |                  |

In [16]:
df.select('ESTACAO').describe().show()


+-------+-------+
|summary|ESTACAO|
+-------+-------+
|  count|  50000|
|   mean|   null|
| stddev|   null|
|    min|   A306|
|    max|   A369|
+-------+-------+



In [17]:
df.select('DATA').describe().show()


+-------+----------+
|summary|      DATA|
+-------+----------+
|  count|     50000|
|   mean|      null|
| stddev|      null|
|    min|2020-01-01|
|    max|2021-01-31|
+-------+----------+



In [18]:
df.select('HORA').describe().show()


+-------+-----------------+
|summary|             HORA|
+-------+-----------------+
|  count|            50000|
|   mean|         1149.872|
| stddev|692.2502205296557|
|    min|                0|
|    max|              900|
+-------+-----------------+



In [19]:
df.select('PRECIPITACAO').describe().show()


+-------+-------------------+
|summary|       PRECIPITACAO|
+-------+-------------------+
|  count|              50000|
|   mean|0.10227919356319039|
| stddev| 1.0429634475200882|
|    min|                   |
|    max|                9.6|
+-------+-------------------+



In [20]:
df.select('PRESSAO_ATMOSFERICA').describe().show()


+-------+-------------------+
|summary|PRESSAO_ATMOSFERICA|
+-------+-------------------+
|  count|              50000|
|   mean|  976.9155263829372|
| stddev|    24.179291972851|
|    min|                   |
|    max|              999.9|
+-------+-------------------+



In [21]:
df.select('TEMPERATURA').describe().show()


+-------+------------------+
|summary|       TEMPERATURA|
+-------+------------------+
|  count|             50000|
|   mean|26.625177204423082|
| stddev| 4.001085447165693|
|    min|                  |
|    max|              38.4|
+-------+------------------+



In [22]:
df.select('UMIDADE').describe().show()


+-------+-----------------+
|summary|          UMIDADE|
+-------+-----------------+
|  count|            50000|
|   mean| 66.7407228605452|
| stddev|19.95580993809345|
|    min|                 |
|    max|             99.0|
+-------+-----------------+



In [23]:
df.select('VENTO').describe().show()

+-------+------------------+
|summary|             VENTO|
+-------+------------------+
|  count|             50000|
|   mean|1.8212265622783852|
| stddev|1.5012775504691624|
|    min|                  |
|    max|               9.5|
+-------+------------------+



In [24]:
float(df.describe("VENTO").filter("summary = 'max'").select("VENTO").collect()[0].asDict()['VENTO'])

9.5

# Percentil dos dados

percentis são medidas que dividem a amostra (por ordem crescente dos dados) em 100 partes, cada uma com uma percentagem de dados aproximadamente igual. 
O k-ésimo percentil Pk é o valor x (xk) que corresponde à frequência cumulativa de N .k/100, onde N é o tamanho amostral. Analise do percentil dos dados para ver o quão proximos estão as camadas de amostras


In [ ]:
#Analise pela hora Hora
df_teste = df.select("ESTACAO","DATA","HORA", F.percent_rank().over(Window.partitionBy().orderBy(df['ESTACAO'])).alias("percent_rank"))
#df_teste.show()
#df_teste.show(10000)


In [ ]:
#Analise pela hora Hora
df_teste = df.select("ESTACAO","DATA","HORA", F.percent_rank().over(Window.partitionBy().orderBy(df["DATA"])).alias("percent_rank"))
#df_teste.show(10000)

In [ ]:
#Analise pela hora Hora
df_teste = df.select("ESTACAO","DATA","HORA", F.percent_rank().over(Window.partitionBy().orderBy(df["HORA"])).alias("percent_rank"))
#df_teste.show(10000)

In [ ]:
#Analise pela hora "TEMPERATURA DO AR - BULBO SECO, HORARIA (C)"
df_teste = df.select("TEMPERATURA","UMIDADE","VENTO", F.percent_rank().over(Window.partitionBy().orderBy(df["TEMPERATURA"])).alias("percent_rank"))
#df_teste.show(10000)

In [ ]:
#Analise pela hora "UMIDADE RELATIVA DO AR, HORARIA (%)"
df_teste = df.select("TEMPERATURA","UMIDADE","VENTO", F.percent_rank().over(Window.partitionBy().orderBy(df["UMIDADE"])).alias("percent_rank"))
#df_teste.show(10000)

In [ ]:
#Analise pela hora Umidade Relativa
df_teste = df.select("TEMPERATURA","UMIDADE","VENTO", F.percent_rank().over(Window.partitionBy().orderBy(df["VENTO"])).alias("percent_rank"))
#df_teste.show(10000)

In [ ]:
#Analise pela hora (Atribtuos da amostra)
df_teste = df.select("ESTACAO","DATA","HORA", "PRECIPITACAO", "TEMPERATURA", "PRESSAO_ATMOSFERICA", "TEMPERATURA", "UMIDADE","VENTO", F.percent_rank().over(Window.partitionBy().orderBy(df["DATA"])).alias("percent_rank"))
df_teste.show(10000)

+-------+----------+----+------------+-----------+-------------------+-----------+-------+-----+--------------------+
|ESTACAO|      DATA|HORA|PRECIPITACAO|TEMPERATURA|PRESSAO_ATMOSFERICA|TEMPERATURA|UMIDADE|VENTO|        percent_rank|
+-------+----------+----+------------+-----------+-------------------+-----------+-------+-----+--------------------+
|   A306|2020-01-01|   0|         0.2|       25.5|             1001.2|       25.5|   77.0|  0.0|                 0.0|
|   A306|2020-01-01| 100|         0.0|       25.1|             1001.6|       25.1|   78.0|  0.0|                 0.0|
|   A306|2020-01-01| 200|         0.0|       25.2|             1001.8|       25.2|   79.0|  0.0|                 0.0|
|   A306|2020-01-01| 300|         0.0|       24.8|             1001.4|       24.8|   80.0|  0.0|                 0.0|
|   A306|2020-01-01| 400|         0.0|       24.6|             1000.6|       24.6|   80.0|  0.0|                 0.0|
|   A306|2020-01-01| 500|         0.2|       24.5|      

# Tratamento de valores misssing/Verificando valores Missing

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-------+----+----+------------+-------------------+-----------+-------+-----+
|ESTACAO|DATA|HORA|PRECIPITACAO|PRESSAO_ATMOSFERICA|TEMPERATURA|UMIDADE|VENTO|
+-------+----+----+------------+-------------------+-----------+-------+-----+
|      0|   0|   0|           0|                  0|          0|      0|    0|
+-------+----+----+------------+-------------------+-----------+-------+-----+



In [ ]:
#Drop e verificação de valores missing se julgado necessário
#df.na.drop()
#df.describe().show()

# Importando bibliotecas para visualização  

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib 

Using matplotlib backend: agg


In [ ]:
#df2 = df
#df_plots = df2.toPandas()
#df_plots.plot()
#display(plt.show())

# Importando bibliotecas de Machine Learning


In [25]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df

DataFrame[ESTACAO: string, DATA: string, HORA: string, PRECIPITACAO: string, PRESSAO_ATMOSFERICA: string, TEMPERATURA: string, UMIDADE: string, VENTO: string]

In [44]:
from pyspark.sql.types import IntegerType
dfnovo = df.withColumn("ESTACAO", df["ESTACAO"].cast("int"))
dfnovo = df.withColumn("DATA", df["DATA"].cast("date"))
dfnovo = df.withColumn("HORA", df["HORA"].cast("int"))
dfnovo = df.withColumn("PRECIPITACAO", df["PRECIPITACAO"].cast("float"))
dfnovo = df.withColumn("PRESSAO_ATMOSFERICA", df["PRESSAO_ATMOSFERICA"].cast("float"))
dfnovo = df.withColumn("TEMPERATURA", df["TEMPERATURA"].cast("float"))
dfnovo = df.withColumn("UMIDADE", df["UMIDADE"].cast("float"))
#dfnovo = df.withColumn("VENTO", df["VENTO"].cast("float"))

In [80]:
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, DateType, FloatType, TimestampType, LongType


#df.withColumn("ESTACAO",df.ESTACAO.cast(IntegerType())).withColumn("DATA",df.DATA.cast(DateType())).withColumn("HORA",df.HORA.cast(IntegerType())).withColumn("PRECIPITACAO",df.PRECIPITACAO.cast(FloatType()))

#.withColumn("PRESSAO_ATMOSFERICA",df.PRESSAO_ATMOSFERICA.cast(FloatType())).withColumn("TEMPERATURA",df.TEMPERATURA.cast(FloatType())).withColumn("UMIDADE",df.UMIDADE.cast(FloatType()))

df = df.withColumn("ESTACAO",df.ESTACAO.cast(IntegerType())).withColumn("DATA",df.DATA.cast(DateType())).withColumn("HORA",df.HORA.cast(IntegerType())).withColumn("PRECIPITACAO",df.PRECIPITACAO.cast(FloatType())).withColumn("PRESSAO_ATMOSFERICA",df.PRESSAO_ATMOSFERICA.cast(FloatType())).withColumn("TEMPERATURA",df.TEMPERATURA.cast(FloatType())).withColumn("UMIDADE",df.UMIDADE.cast(FloatType()))

df = df.withColumn("DATA",df.DATA.cast(IntegerType()))

df
# dfnovo.withColumn("ESTACAO",df.ESTACAO.cast(IntegerType())).withColumn("DATA",df.DATA.cast(DateType()))

# .withColumn("HORA",df.HORA.cast(IntegerType()))
# .withColumn("PRECIPITACAO",df.PRECIPITACAO.cast(FloatType()))
# .withColumn("PRESSAO_ATMOSFERICA",df.PRESSAO_ATMOSFERICA.cast(FloatType()))
# .withColumn("TEMPERATURA",df.TEMPERATURA.cast(FloatType()))
# .withColumn("UMIDADE",df.UMIDADE.cast(FloatType()))

#atmosfera



# dfnovo.withColumn("DATA",df.DATA.cast(DateType()))
# dfnovo.withColumn("HORA",df.HORA.cast(IntegerType()))
#df.withColumn("age",df.age.cast(IntegerType()))


DataFrame[ESTACAO: int, DATA: int, HORA: int, PRECIPITACAO: float, PRESSAO_ATMOSFERICA: float, TEMPERATURA: float, UMIDADE: float, VENTO: string]

In [69]:
df

DataFrame[ESTACAO: int, DATA: date, HORA: int, PRECIPITACAO: float, PRESSAO_ATMOSFERICA: float, TEMPERATURA: float, UMIDADE: float, VENTO: string]

In [ ]:
changedTypedf.show()

+-------+----------+----+------------+-------------------+-----------+-------+-----+
|ESTACAO|      DATA|HORA|PRECIPITACAO|PRESSAO_ATMOSFERICA|TEMPERATURA|UMIDADE|VENTO|
+-------+----------+----+------------+-------------------+-----------+-------+-----+
|   A306|2021-01-01|0000|         0.0|             1000.1|       27.7|   65.0|  3.4|
|   A306|2021-01-01|0100|         0.0|             1000.9|       27.0|   74.0|  0.6|
|   A306|2021-01-01|0200|         0.0|             1001.5|       26.3|   79.0|  0.0|
|   A306|2021-01-01|0300|         0.0|             1001.2|       26.1|   82.0|  0.0|
|   A306|2021-01-01|0400|         0.0|             1000.7|       26.0|   84.0|  0.3|
|   A306|2021-01-01|0500|         0.0|             1000.3|       25.6|   84.0|  0.0|
|   A306|2021-01-01|0600|         0.0|             1000.0|       25.1|   87.0|  0.0|
|   A306|2021-01-01|0700|         0.0|             1000.1|       24.8|   88.0|  0.0|
|   A306|2021-01-01|0800|         0.0|             1000.9|       

In [51]:
def plot_corr(df, size=20):
    corr = df.corr()    
    fig, ax = plt.subplots(figsize = (size, size))
    ax.matshow(corr)  
    plt.xticks(range(len(corr.columns)), corr.columns) 
    plt.yticks(range(len(corr.columns)), corr.columns)

In [52]:
#print do grafico gerado
plot_corr(df)

TypeError: ignored

Bibliotecas Sklearn

In [53]:
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn 
print (sklearn.__version__)
import numpy as np
from sklearn.impute import SimpleImputer

0.22.2.post1


In [ ]:
# Criando objeto para substituição dos valores que faltam pela media, eles serao posterirmente inseridos dessa maneira
# nos dados de treino

In [54]:
# Criando objeto para substituição dos valores que faltam pela media, eles serao posterirmente inseridos dessa maneira
# nos dados de treino
preenche = SimpleImputer(missing_values = 0, strategy = "mean")

In [55]:
# Variável a ser prevista
atrib_prev = ['VENTO']

In [56]:
# Seleção de variáveis preditoras (Feature Selection)
atributos = ['DATA', 'HORA', 'PRECIPITACAO', 'PRESSAO_ATMOSFERICA', 'TEMPERATURA', 'UMIDADE']

In [58]:
# Criando objetos
#Colunas desejadas 
#1 ESTACAO| 2 DATA| 3 HORA| 4 PRECIPITACAO|5 PRESSAO_ATMOSFERICA| 6 TEMPERATURA| 7 UMIDADE| 8 VENTO
def func(lista):
  return lista[1], lista[2], lista[3], lista[4], lista[5], lista[6], lista[7]

def func2(lista):
  return lista[8]
##print(df[["ESTACAO","HORA"]])
#x = df.map(func)#"DATA", 'PRECIPITACAO', 'PRESSAO_ATMOSFERICA', 'TEMPERATURA', 'UMIDADE"

#X = df.select(atributos)
#Y = df.select(atrib_prev)

X = df.select(atributos).collect()
Y = df.select(atrib_prev).collect()


In [62]:
Y[10]

Row(VENTO='0.0')

In [59]:
#Print dos dados dentro armazenados Y
print(Y.take(100))
#y.take(100))
#print(Y.collect())

AttributeError: ignored

In [ ]:
#Print dos dados dentro armazenados X
print(X.take(100))
#X.take(100))
#print(X.collect())

[Row(DATA='2021-01-01', HORA='0000', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.1', TEMPERATURA='27.7', UMIDADE='65.0'), Row(DATA='2021-01-01', HORA='0100', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.9', TEMPERATURA='27.0', UMIDADE='74.0'), Row(DATA='2021-01-01', HORA='0200', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1001.5', TEMPERATURA='26.3', UMIDADE='79.0'), Row(DATA='2021-01-01', HORA='0300', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1001.2', TEMPERATURA='26.1', UMIDADE='82.0'), Row(DATA='2021-01-01', HORA='0400', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.7', TEMPERATURA='26.0', UMIDADE='84.0'), Row(DATA='2021-01-01', HORA='0500', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.3', TEMPERATURA='25.6', UMIDADE='84.0'), Row(DATA='2021-01-01', HORA='0600', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.0', TEMPERATURA='25.1', UMIDADE='87.0'), Row(DATA='2021-01-01', HORA='0700', PRECIPITACAO='0.0', PRESSAO_ATMOSFERICA='1000.1', TEMPERATURA='24.8', UMIDADE='88.0'), Row(DATA='2021-

Fazendo um split, uma divisão de dados para serem treinados

In [ ]:
# Definindo a taxa de split em 30% e 70% como explicados em sala de aula e debatidos
split_test_size = 0.30

In [61]:
# Criando um modelo de dados para realizar o treino e de teste
X_treino, X_teste, Y_treino, Y_teste = train_test_split(X, Y, test_size=0.30, random_state=101)

In [67]:
Y_teste

[Row(VENTO='4.1'),
 Row(VENTO='0.0'),
 Row(VENTO='1.9'),
 Row(VENTO='0.0'),
 Row(VENTO='3.1'),
 Row(VENTO='2.3'),
 Row(VENTO=''),
 Row(VENTO='0.8'),
 Row(VENTO='3.0'),
 Row(VENTO=''),
 Row(VENTO='1.4'),
 Row(VENTO='0.0'),
 Row(VENTO='2.6'),
 Row(VENTO='2.1'),
 Row(VENTO=''),
 Row(VENTO=''),
 Row(VENTO='5.1'),
 Row(VENTO=''),
 Row(VENTO='2.9'),
 Row(VENTO=''),
 Row(VENTO='1.2'),
 Row(VENTO='2.3'),
 Row(VENTO=''),
 Row(VENTO=''),
 Row(VENTO='0.8'),
 Row(VENTO=''),
 Row(VENTO='2.2'),
 Row(VENTO='0.0'),
 Row(VENTO='3.5'),
 Row(VENTO='2.1'),
 Row(VENTO='2.7'),
 Row(VENTO=''),
 Row(VENTO='3.0'),
 Row(VENTO=''),
 Row(VENTO='2.7'),
 Row(VENTO=''),
 Row(VENTO='4.5'),
 Row(VENTO='0.0'),
 Row(VENTO='0.1'),
 Row(VENTO='1.0'),
 Row(VENTO='2.0'),
 Row(VENTO='2.6'),
 Row(VENTO='0.5'),
 Row(VENTO=''),
 Row(VENTO='0.0'),
 Row(VENTO='1.4'),
 Row(VENTO='1.0'),
 Row(VENTO=''),
 Row(VENTO='2.6'),
 Row(VENTO='4.2'),
 Row(VENTO='3.7'),
 Row(VENTO='1.1'),
 Row(VENTO='2.6'),
 Row(VENTO=''),
 Row(VENTO='0.0'),


#### Apos a separação dos dados de treino e teste para o modelo. 
- Após criar os dados de treino e teste  do modelo

In [63]:
# Imprimindo os resultados de como os dados serão treinados e testados
#print("{0:0.2f}% nos dados de treino".format((len(X_treino)/len(df.index)) * 100))
#print("{0:0.2f}% nos dados de teste".format((len(X_teste)/len(df.index)) * 100))

AttributeError: ignored

In [ ]:
from sklearn.model_selection import train_test_split